### 배치단위로 padding하는 방법(dynamic padding) 1번째

In [ ]:
!git clone https://github.com/e9t/nsmc.git  ### naver 뉴스댓글

fatal: Too many arguments.

usage: git clone [<options>] [--] <repo> [<dir>]

    -v, --verbose         be more verbose
    -q, --quiet           be more quiet
    --progress            force progress reporting
    -n, --no-checkout     don't create a checkout
    --bare                create a bare repository
    --mirror              create a mirror repository (implies bare)
    -l, --local           to clone from a local repository
    --no-hardlinks        don't use local hardlinks, always copy
    -s, --shared          setup as shared repository
    --recursive ...       alias of --recurse-submodules
    --recurse-submodules[=<pathspec>]
                          initialize submodules in the clone
    -j, --jobs <n>        number of submodules cloned in parallel
    --template <template-directory>
                          directory from which templates will be used
    --reference <repo>    reference repository
    --reference-if-able <repo>
                          reference re

In [1]:
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [2]:
import pandas as pd
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

train = train[~pd.isnull(train['document'])]
train_text = train['document'].values
labels = train.label.tolist()
train_text = train_text.tolist()

In [3]:
from transformers import AutoModel, AutoTokenizer, BertTokenizer
MODEL_NAME = 'distilbert-base-uncased'

model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
## input to ids 만 하는것

full_input_ids = []

# For each training example...
for text in train_text:

    # Tokenize the sentence.
    input_ids = tokenizer.encode(text=text,           # Movie review text
                                 add_special_tokens=True, # Do add specials.
                                #  max_length=max_len,  # Do truncate to `max_len`
                                #  truncation=True,     # Do truncate!
                                 padding=False)       # Don't pad!
                                 
    # Add the tokenized result to our list.
    full_input_ids.append(input_ids)
    
print('DONE.')

DONE.


In [5]:
### 한 단계 더 나아가서 input 길이를 sorting해서 작은것끼리 배치로 묶는 방법 사용. 
### 이는 블로그를 참고했고, 작은 길이는 작은길이끼리 배치로 묶는게 컴퓨터 리소스 적게 먹는다는 아이디어. 

train_samples = sorted(zip(full_input_ids, labels), key=lambda x: len(x[0]))

In [6]:
### 배치 자체를 만드는 코드

import random

batch_size = 16
# List of batches that we'll construct.
batch_ordered_sentences = []
batch_ordered_labels = []

print('Creating training batches of size {:}'.format(batch_size))

# Loop over all of the input samples...    
while len(train_samples) > 0:
    
    # Report progress.
    if ((len(batch_ordered_sentences) % 500) == 0):
        print('  Selected {:,} batches.'.format(len(batch_ordered_sentences)))

    # `to_take` is our actual batch size. It will be `batch_size` until 
    # we get to the last batch, which may be smaller. 
    to_take = min(batch_size, len(train_samples))

    # Pick a random index in the list of remaining samples to start
    # our batch at.
    select = random.randint(0, len(train_samples) - to_take)

    # Select a contiguous batch of samples starting at `select`.
    batch = train_samples[select:(select + to_take)]

    # Each sample is a tuple--split them apart to create a separate list of 
    # sequences and a list of labels for this batch.
    batch_ordered_sentences.append([s[0] for s in batch])
    batch_ordered_labels.append([s[1] for s in batch])

    # Remove these samples from the list.
    del train_samples[select:select + to_take]

print('\n  DONE - {:,} batches.'.format(len(batch_ordered_sentences)))

Creating training batches of size 16
  Selected 0 batches.
  Selected 500 batches.
  Selected 1,000 batches.
  Selected 1,500 batches.
  Selected 2,000 batches.
  Selected 2,500 batches.
  Selected 3,000 batches.
  Selected 3,500 batches.
  Selected 4,000 batches.
  Selected 4,500 batches.
  Selected 5,000 batches.
  Selected 5,500 batches.
  Selected 6,000 batches.
  Selected 6,500 batches.
  Selected 7,000 batches.
  Selected 7,500 batches.
  Selected 8,000 batches.
  Selected 8,500 batches.
  Selected 9,000 batches.

  DONE - 9,375 batches.


In [7]:
### 만든 배치를 이용해서 배치단위로 padding하는 것. 
### 이렇게 하기 귀찮아서..그냥 고정길이로 다들 하는건가...
### 배치단위로 이렇게 PADDING 하기 위해서 FOR문을 사용하는 것 여긴..
 
import torch

py_inputs = []
py_attn_masks = []
py_labels = []

# For each batch...
for (batch_inputs, batch_labels) in zip(batch_ordered_sentences, batch_ordered_labels):

    # New version of the batch, this time with padded sequences and now with
    # attention masks defined.
    batch_padded_inputs = []
    batch_attn_masks = []
    
    # First, find the longest sample in the batch. 
    # Note that the sequences do currently include the special tokens!
    max_size = max([len(sen) for sen in batch_inputs])

    #print('Max size:', max_size)

    # For each input in this batch...
    for sen in batch_inputs:
        
        # How many pad tokens do we need to add?
        num_pads = max_size - len(sen)

        # Add `num_pads` padding tokens to the end of the sequence.
        padded_input = sen + [tokenizer.pad_token_id]*num_pads

        # Define the attention mask--it's just a `1` for every real token
        # and a `0` for every padding token.
        attn_mask = [1] * len(sen) + [0] * num_pads

        # Add the padded results to the batch.
        batch_padded_inputs.append(padded_input)
        batch_attn_masks.append(attn_mask)

    # Our batch has been padded, so we need to save this updated batch.
    # We also need the inputs to be PyTorch tensors, so we'll do that here.
    py_inputs.append(torch.tensor(batch_padded_inputs))
    py_attn_masks.append(torch.tensor(batch_attn_masks))
    py_labels.append(torch.tensor(batch_labels))


In [8]:
print(py_inputs[0].shape)
print(py_inputs[1].shape)
print(py_inputs[2].shape)

torch.Size([16, 50])
torch.Size([16, 205])
torch.Size([16, 48])


In [9]:
from transformers import AutoModelForSequenceClassification 

model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path='distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)

In [19]:
### 이렇게 배치간 길이가 달라도 학습이 가능하다는것을 증명!!!

model.train()
optimizer.zero_grad()
logits = model(py_inputs[0], attention_mask=py_attn_masks[0])
loss = torch.nn.functional.nll_loss(logits['logits'], py_labels[0])
loss.backward()
optimizer.step()

optimizer.zero_grad()
logits = model(py_inputs[1], attention_mask=py_attn_masks[1])
loss = torch.nn.functional.nll_loss(logits['logits'], py_labels[1])
print(loss)
loss.backward()
optimizer.step()
 

tensor(-15.2023, grad_fn=<NllLossBackward0>)


In [10]:
### 이런식으로 내가 직접 input_ids, attention_mask dictionary 형태로 만들어서 집어넣는것도 가능하다. 
model(**{'input_ids' : py_inputs[0], 'attention_mask' : py_attn_masks[0]})

SequenceClassifierOutput(loss=None, logits=tensor([[0.1566, 0.2231],
        [0.1555, 0.2490],
        [0.1566, 0.2217],
        [0.1744, 0.2401],
        [0.1794, 0.2326],
        [0.1527, 0.2560],
        [0.1704, 0.2244],
        [0.1468, 0.2280],
        [0.1678, 0.2448],
        [0.1381, 0.2351],
        [0.1279, 0.2492],
        [0.1617, 0.2286],
        [0.1621, 0.2316],
        [0.1741, 0.2248],
        [0.1664, 0.2176],
        [0.1614, 0.2449]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [11]:
### 더불어 이런식으로 여러개 tensor를 한번에 넣을때는 이렇게 넣으면되는것. 
py_inputs[0], py_attn_masks[0]  

(tensor([[  101,   100,  1464, 30019, 30024, 30000, 30014, 30025, 29994, 30010,
          30020, 29999, 30019,  1469, 30017, 30003, 30017, 29994, 30008, 30000,
          30019, 30000, 30019,   100,   100,  1463, 30010, 30021, 29991, 30019,
          29999, 30012,   100,  1455, 30010, 30022, 29995, 30006, 30022, 29993,
          30011,   100,  1460, 30011, 30024, 29992, 30019, 29993, 30006,   102],
         [  101,  1456, 30006,  1464, 30006, 29997, 30019, 30021, 29999, 30017,
          30022,  1457, 30011, 30022, 29999, 30019, 30002, 30010, 29996, 30011,
          29991, 30009, 30005, 30006, 29991, 30011, 29996, 30006, 30021, 29997,
          30008, 30025, 30005, 30006, 29991, 30009,  1459, 30006, 30021, 29993,
          30017, 29992, 30017, 30021,  1463, 30010, 30025, 30005, 30012,   102],
         [  101,  1005,  1464, 30014, 29996, 30014,  1005,  1458, 30006, 29992,
          30017, 30021,   100,  1457, 30007, 30005, 30007,  1457, 30006, 29997,
          30019, 30005, 30006, 30021, 

In [ ]:
### 이건 참고로, 모든 sequence의 길이를 통일하는 방법. 이렇게 하면 모든 batch가 동일한 길이로 padding되는 듯하다. 

use_fixed_padding = True

if use_fixed_padding:

    # Specify batch_size and truncation length.    
    batch_size = 16
    max_len = 400   

    # Tokenize all training examples
    print('Tokenizing {:,} training samples...'.format(len(train_text)))

    # Tokenize all of the sentences and map the tokens to thier word IDs.
    batches_input_ids = []
    batches_attention_masks = []
    batches_labels = []

    update_interval = batch_size * 150 

    # For every sentence...
    for i in range(0, len(train_text), batch_size):

        # Report progress.
        if ((i % update_interval) == 0):
            print('  Tokenized {:,} samples.'.format(i))

        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.batch_encode_plus(
                            train_text[i:i+batch_size], # Batch of sentences to encode.
                            add_special_tokens = True,  # Add '[CLS]' and '[SEP]'
                            max_length = 400,           # Pad & truncate all sentences.
                            padding = 'max_length',     # Pad all to the `max_length` parameter.
                            truncation = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                    )
        
        # Add the encoded sentence to the list.    
        batches_input_ids.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        batches_attention_masks.append(encoded_dict['attention_mask'])

        # Add the labels for the batch
        batches_labels.append(torch.tensor(labels[i:i+batch_size]))
    
    # Rename the final variable to match the rest of the code in this Notebook.
    py_inputs = batches_input_ids
    py_attn_masks = batches_attention_masks
    py_labels = batches_labels

Tokenizing 149,995 training samples...
  Tokenized 0 samples.
  Tokenized 2,400 samples.
  Tokenized 4,800 samples.
  Tokenized 7,200 samples.
  Tokenized 9,600 samples.
  Tokenized 12,000 samples.
  Tokenized 14,400 samples.
  Tokenized 16,800 samples.
  Tokenized 19,200 samples.
  Tokenized 21,600 samples.
  Tokenized 24,000 samples.
  Tokenized 26,400 samples.
  Tokenized 28,800 samples.
  Tokenized 31,200 samples.
  Tokenized 33,600 samples.
  Tokenized 36,000 samples.
  Tokenized 38,400 samples.
  Tokenized 40,800 samples.
  Tokenized 43,200 samples.
  Tokenized 45,600 samples.
  Tokenized 48,000 samples.
  Tokenized 50,400 samples.
  Tokenized 52,800 samples.
  Tokenized 55,200 samples.
  Tokenized 57,600 samples.
  Tokenized 60,000 samples.
  Tokenized 62,400 samples.
  Tokenized 64,800 samples.
  Tokenized 67,200 samples.
  Tokenized 69,600 samples.
  Tokenized 72,000 samples.
  Tokenized 74,400 samples.
  Tokenized 76,800 samples.
  Tokenized 79,200 samples.
  Tokenized 81,600

In [ ]:
print(py_inputs[0].shape)
print(py_inputs[1].shape)
print(py_inputs[2].shape)

torch.Size([16, 400])
torch.Size([16, 400])
torch.Size([16, 400])


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    pretrained_model_name_or_path='distilbert-base-uncased')

## distilbert 자체는 학습하지 않도록. 

for param in model.distilbert.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'pre_classifi

In [ ]:
### 학습 전 가중치 확인. 
print(model.classifier.state_dict()['weight'])
print(model.distilbert.state_dict()['embeddings.word_embeddings.weight'])
print(model.distilbert.state_dict()['embeddings.LayerNorm.weight'])

tensor([[-0.0221,  0.0464, -0.0018,  ..., -0.0121,  0.0260,  0.0291],
        [ 0.0165,  0.0118, -0.0105,  ...,  0.0240, -0.0199, -0.0137]])
tensor([[-0.0166, -0.0666, -0.0163,  ..., -0.0200, -0.0514, -0.0264],
        [-0.0132, -0.0673, -0.0161,  ..., -0.0227, -0.0554, -0.0260],
        [-0.0176, -0.0709, -0.0144,  ..., -0.0246, -0.0596, -0.0232],
        ...,
        [-0.0231, -0.0588, -0.0105,  ..., -0.0195, -0.0262, -0.0212],
        [-0.0490, -0.0561, -0.0047,  ..., -0.0107, -0.0180, -0.0219],
        [-0.0065, -0.0915, -0.0025,  ..., -0.0151, -0.0504,  0.0460]])
tensor([0.8425, 0.8005, 0.7313, 0.7595, 0.7978, 0.7835, 0.8177, 0.8189, 0.8398,
        0.7094, 0.6675, 0.6861, 0.8101, 0.6887, 0.8038, 0.7560, 0.7175, 0.7307,
        0.8399, 0.7757, 0.8152, 0.7324, 0.8066, 0.7566, 0.7473, 0.8094, 0.6373,
        0.8413, 0.7010, 0.7445, 0.8248, 0.7532, 0.8007, 0.8824, 0.8201, 0.7667,
        0.8325, 0.7654, 0.8201, 0.7824, 0.8443, 0.8523, 0.7780, 0.7448, 0.7763,
        0.7377, 0.6849, 0

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 5e-5, # This is the value Michael used.
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

# Number of training epochs. I chose to train for 1 simply because the training
# time is long. More epochs may improve the model's accuracy.
epochs = 1

# Total number of training steps is [number of batches] x [number of epochs]. 
# Note that it's the number of *batches*, not *samples*!
total_steps = len(py_inputs) * epochs

# Create the learning rate scheduler.
scheduler = get_cosine_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
     
    print('Training on {:,} batches...'.format(len(py_inputs)))
  

    # Reset the total loss for this epoch.
    total_train_loss = 0
    
    model.train()

    # For each batch of training data...
    for step in range(0, 10): ## len(py_inputs) -- 일부러 10개로만 해보자. 
      
        print(step, 10)
        # Copy the current training batch to the GPU using the `to` method.
        b_input_ids = py_inputs[step]
        b_input_mask = py_attn_masks[step]
        b_labels = py_labels[step]

        # Always clear any previously calculated gradients before performing a
        # backward pass.
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The call returns the loss (because we provided labels) and the 
        # "logits"--the model outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)['loss'], model(b_input_ids, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)['logits']

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(py_inputs)            
    


======== Epoch 1 / 1 ========
Training on 9,375 batches...
0 10
1 10
2 10
3 10
4 10
5 10
6 10
7 10
8 10
9 10


In [ ]:
### 학습 후 parameter 변화.. distilbert는 변하지 않았다. 
print(model.classifier.state_dict()['weight'])
print(model.distilbert.state_dict()['embeddings.word_embeddings.weight'])
print(model.distilbert.state_dict()['embeddings.LayerNorm.weight'])

tensor([[-0.0222,  0.0461, -0.0020,  ..., -0.0120,  0.0259,  0.0290],
        [ 0.0167,  0.0121, -0.0104,  ...,  0.0240, -0.0197, -0.0135]])
tensor([[-0.0166, -0.0666, -0.0163,  ..., -0.0200, -0.0514, -0.0264],
        [-0.0132, -0.0673, -0.0161,  ..., -0.0227, -0.0554, -0.0260],
        [-0.0176, -0.0709, -0.0144,  ..., -0.0246, -0.0596, -0.0232],
        ...,
        [-0.0231, -0.0588, -0.0105,  ..., -0.0195, -0.0262, -0.0212],
        [-0.0490, -0.0561, -0.0047,  ..., -0.0107, -0.0180, -0.0219],
        [-0.0065, -0.0915, -0.0025,  ..., -0.0151, -0.0504,  0.0460]])
tensor([0.8425, 0.8005, 0.7313, 0.7595, 0.7978, 0.7835, 0.8177, 0.8189, 0.8398,
        0.7094, 0.6675, 0.6861, 0.8101, 0.6887, 0.8038, 0.7560, 0.7175, 0.7307,
        0.8399, 0.7757, 0.8152, 0.7324, 0.8066, 0.7566, 0.7473, 0.8094, 0.6373,
        0.8413, 0.7010, 0.7445, 0.8248, 0.7532, 0.8007, 0.8824, 0.8201, 0.7667,
        0.8325, 0.7654, 0.8201, 0.7824, 0.8443, 0.8523, 0.7780, 0.7448, 0.7763,
        0.7377, 0.6849, 0

In [ ]:
model.eval()

with torch.no_grad():
    
    logits = model(**tokenizer(train.document[8999], return_tensors='pt'))['logits'][0]
    prediction = torch.nn.functional.softmax(logits).argmax(dim=-1)
    print(logits)
    print(prediction)
    print(torch.nn.functional.softmax(logits))

tensor([-0.1385,  0.1663])
tensor(1)
tensor([0.4244, 0.5756])


C:\Users\chkim\AppData\Local\Temp\ipykernel_16728\3315986766.py:6: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prediction = torch.nn.functional.softmax(logits).argmax(dim=-1)
C:\Users\chkim\AppData\Local\Temp\ipykernel_16728\3315986766.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(torch.nn.functional.softmax(logits))


---
### 배치별 길이 다르게 padding 하는 방법 2번째 : Dataset, Dataloader 사용

In [12]:
import pandas as pd
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

In [13]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        self.train = pd.read_csv('nsmc/ratings_train.txt', sep = '\t')
        self.train = self.train[~pd.isnull(self.train.document)]
        self.text = self.train.document.values.tolist()
        self.label = self.train.label.values.tolist()
    def __getitem__(self, i):
        return self.text[i], self.label[i]

    def __len__(self):
        return len(self.label)

In [14]:
dataset = CustomDataset()
next(iter(dataset))

('아 더빙.. 진짜 짜증나네요 목소리', 0)

In [15]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer
MODEL_NAME = 'distilbert-base-uncased'

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

In [16]:
### collate_fn을 만들자. 
### collate_fn은 dataloader 와 dataset 사이에 있다고 생각하면된다. dataset에서 넘어온것들이 collate_fn을 거친다. 
def make_batch_pad(samples):
    inputs = tokenizer([sample[0] for sample in samples], padding=True, add_special_tokens=True, return_tensors='pt')
    labels = torch.LongTensor([sample[1] for sample in samples])
    
    return inputs, labels

In [17]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 4, collate_fn = make_batch_pad)

In [22]:
## 배치단위로 padding이 되었다. 
cnt=0
for input, label in dataloader:
    print(input['input_ids'].shape)
    print(label)
    cnt+=1
    if cnt==4:
        break

torch.Size([4, 56])
tensor([0, 1, 0, 0])
torch.Size([4, 101])
tensor([1, 0, 0, 0])
torch.Size([4, 67])
tensor([1, 1, 1, 1])
torch.Size([4, 99])
tensor([0, 1, 0, 1])


In [ ]:
model(**input)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.0420, -0.0604],
        [-0.0432, -0.0567],
        [ 0.0328, -0.0197],
        [-0.0486, -0.0581]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)